In [ ]:
#ZADANIE 1


In [1]:
# ZADANIE 2
import numpy as np

nn_architecture = [
    {"input_dim": 2, "output_dim": 2, "activation": "sigmoid"},
    {"input_dim": 2, "output_dim": 1, "activation": "sigmoid"},
]

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def init_layers(nn_architecture, seed=99):
    np.random.seed(seed)
    params_values = {}
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) * 0.1
        params_values['b' + str(layer_idx)] = np.random.randn(
            layer_output_size, 1) * 0.1
    return params_values

def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="sigmoid"):
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    A_curr = sigmoid(Z_curr)
    return A_curr, Z_curr

def full_forward_propagation(X, params_values, nn_architecture):
    memory = {}
    A_curr = X
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        A_prev = A_curr
        W_curr = params_values["W" + str(layer_idx)]
        b_curr = params_values["b" + str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activation=layer["activation"])
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
    return A_curr, memory

def get_cost_value(Y_hat, Y):
    m = Y_hat.shape[1]
    cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

def single_layer_backward_propagation(dA_curr, W_curr, Z_curr, A_prev, activation="sigmoid"):
    m = A_prev.shape[1]
    dZ_curr = sigmoid_backward(dA_curr, Z_curr)
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)
    return dA_prev, dW_curr, db_curr

def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat))
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_idx_curr = layer_idx_prev + 1
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        W_curr = params_values["W" + str(layer_idx_curr)]
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_prev, W_curr, Z_curr, A_prev, activation=layer["activation"])
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr
    return grads_values

X = np.random.randn(2, 5)
Y = (np.random.randn(1, 5) > 0).astype(int)

params_values = init_layers(nn_architecture)

Y_hat, memory = full_forward_propagation(X, params_values, nn_architecture)

cost = get_cost_value(Y_hat, Y)
print(f"Koszt: {cost}")

grads_values = full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture)

for key in grads_values:
    print(f"{key}:\n{grads_values[key]}\n")


Koszt: 0.7138296834884063
dW2:
[[0.15498574 0.15864785]]

db2:
[[0.31641286]]

dW1:
[[ 0.0045933  -0.0003582 ]
 [ 0.00509324 -0.00046563]]

db1:
[[0.00590026]
 [0.00650342]]

